In [1]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

cache_dir = "/mnt/sdb1tb/ege/ceng502"

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch32-224-in21k', cache_dir=cache_dir)
model = ViTModel.from_pretrained('google/vit-base-patch32-224-in21k', cache_dir=cache_dir)

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state

2023-05-01 14:03:55.126202: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 14:04:02.879173: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-01 14:04:03.013326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-01 14:04:03.013350: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [71]:
outputs.pooler_output.shape

torch.Size([1, 768])

In [50]:
last_hidden_state.shape

torch.Size([1, 50, 768])

In [36]:
model.encoder.layer[0].attention.attention.query

Linear(in_features=768, out_features=768, bias=True)

In [37]:
model.encoder.layer[0]

ViTLayer(
  (attention): ViTAttention(
    (attention): ViTSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (output): ViTSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (intermediate): ViTIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): ViTOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm_before): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (layernorm_after): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
)

In [41]:
for name, param in model.named_parameters():
    print(name)
    if name == "embeddings.cls_token":
        cls_token = param
    

embeddings.cls_token
embeddings.position_embeddings
embeddings.patch_embeddings.projection.weight
embeddings.patch_embeddings.projection.bias
encoder.layer.0.attention.attention.query.weight
encoder.layer.0.attention.attention.query.bias
encoder.layer.0.attention.attention.key.weight
encoder.layer.0.attention.attention.key.bias
encoder.layer.0.attention.attention.value.weight
encoder.layer.0.attention.attention.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.layernorm_before.weight
encoder.layer.0.layernorm_before.bias
encoder.layer.0.layernorm_after.weight
encoder.layer.0.layernorm_after.bias
encoder.layer.1.attention.attention.query.weight
encoder.layer.1.attention.attention.query.bias
encoder.layer.1.attention.attention.key.weight
encoder.layer.1.attention.attention

In [44]:
cls_token.requires_grad

True

In [62]:
last_hidden_state[:,0]

tensor([[ 1.4724e-01,  1.0381e-01, -4.9984e-02, -1.0894e-01,  2.0840e-01,
         -5.9280e-02, -6.7109e-02, -1.2395e-01,  3.5361e-01,  3.1208e-01,
          1.0147e-01,  3.2999e-02, -5.0150e-03, -8.4950e-02,  1.9849e-01,
         -8.2933e-02,  8.3284e-04, -7.4282e-02,  2.0753e-01, -6.9254e-02,
          8.6422e-02,  3.1460e-01,  5.1740e-01,  3.3572e-02,  5.0058e-02,
         -1.0896e-01, -1.2060e-01,  9.1437e-02, -5.8499e-02,  2.3047e-01,
         -2.3624e-01, -4.0661e-03, -1.5772e-01, -3.6367e-02, -3.4246e-01,
         -7.6734e-02, -1.3804e-01, -6.3713e-02,  1.9820e-01, -1.1941e-04,
          7.7860e-03, -1.6803e-02, -1.7929e-01, -2.2642e-01,  1.0151e-01,
         -5.7987e-02, -5.5334e-02,  1.6411e-01, -4.0961e-02, -1.7172e-01,
         -2.2321e-02, -5.6321e-02, -3.8074e-01, -3.5759e-01, -7.1236e-02,
          2.8638e-01, -8.9613e-02, -2.3077e-01, -1.7144e-01,  2.5845e-01,
         -6.5910e-02, -1.6854e-02, -1.1071e-01, -3.7961e-01,  8.9016e-03,
         -6.9926e-03, -2.0502e-01,  9.

ViTPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)